In [6]:
#%pip install nltk
#%pip install pandas
#%pip install numpy
%pip install scikit-learn
#%pip install spacy


  Using cached scikit_learn-1.3.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.11.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB 653.6 kB/s eta 0:00:15
   ---------------------------------------- 0.1/9.3 MB 469.7 kB/s eta 0:00:20
   ---------------------------------------- 0.1/9.3 MB 469.7 kB/s eta 0:00:20
   ---------------------------------------- 0.1/9.3 MB 357.2 kB/s eta 0:00:26
   ---------------------------------------- 0.1/9.3 MB 357.2 kB/s eta 0:00:26
   ---------------------------------------- 0.1/9.3 MB 262.6 kB/s eta 0:00:35
   ---------------------------------------- 0.1/9.3 MB 262.6 kB/s eta 0:00:35
   ---------------------------------------- 0.1/9.3 MB 285.2 kB/s eta 0:00:33
   ---------------

In [7]:
import nltk
from nltk import sent_tokenize
from nltk import wordpunct_tokenize
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn

In [11]:
test_ds1_path = "C:\\course_work_git\\feature_engineere_NLP\\datasets\\test_ds1\\train.jsonl"

In [12]:
data = pd.read_json(test_ds1_path, lines=True)
data

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
0,Сумма ущерба составила одну тысячу рублей. Уто...,entailment,Ранее местный житель совершал подобное правона...,судить,no_negation,kp,0,NaN
1,"Перебрасываясь словечками, они скользят глазам...",contradiction,Они что-то понимают,смотреть,no_negation,fiction,1,NaN
2,"— Разве что, — сказала она, — мы хотим где-ниб...",neutral,"Это “ Таганская ”, а не “ Тульская ”.",понять,no_negation,fiction,2,NaN
3,"Зима, наконец, показала свой характер.",contradiction,У зимы есть свой характер.,показать,no_negation,kp,3,NaN
4,ГуманностьБогаподобнымсценариемнепредполагаетс...,contradiction,Это должно быть просто.,сказать,no_negation,fiction,4,NaN
...,...,...,...,...,...,...,...,...
433,Тверские спортсмены показали отличный результа...,entailment,У тверских спортсменов пять новых медалей.,суметь,no_negation,kp,433,NaN
434,"Просто я не хочу с ним жить и, извини, больше ...",contradiction,Этого было вполне достаточно.,уверять,no_negation,fiction,434,NaN
435,"Второй аргумент: все читаем и видим, что надви...",neutral,Надвигается вторая волна кризиса.,видеть,negation,interfax,435,NaN
436,"— Да, сын. Здравствуй. — Ты можешь говорить, н...",neutral,Сын не занят,говорить,no_negation,fiction,436,NaN
